# MySQL 实战45讲

案例：比如你有一个表，只有一个ID字段，执行如下的查询语句：

`myswql> select * from T where ID=10;`

## 基础架构：一条SQL查询语句是如何执行的？

MySQL分为server层和存储引擎层两个部分：
* server层包括连接器、查询缓存、分析器、优化器、执行器等，所有跨存储引擎的功能都在这个层面实现；
* 存储引擎层负责数据的存储和提取。架构为插件式的，支持InnoDB、MyISAM、Memory等多个存储引擎，最常用的是InnoDB。

下图是**一条查询语句的执行过程**，可以看出不同的存储引擎共用一个**Server层**

<img src="figure/mysql架构.jpg" width="400" hegiht="300" align=left />

### 连接器

#### 连接器的定义和特性

连接器负建立连接、获取权限、维持和管理连接，一般连接命令如下：`mysql -u root -p`,不建议在`-p`后面直接输入密码，容易造成密码泄露。
1. 一般连接成功后，连接器会到权限表里查出你拥有的权限。如果管理员账号对此用户的权限进行修改，也不会影响已经存在的连接，下次重新登录mysql才会生效；
2. 如果连接成功之后，客户端太长时间没有交互和操作，连接器会自动断开，时间由参数wait_timeout控制，默认是8小时；

#### 关于**长连接和短连接**

**1.定义**

**长连接**是指连接成功后，客户端持续有请求，一直使用同一个连接；**短连接**是指每次只需玩很少的几次查询之后就断开连接，下次查询再重新建立。

<a>_**一个比方：如果把mysql的server比作一个洗澡的以人的体量为单位的热水池，将每次客户端请求当做来洗澡的客人，长连接相当于一直给热水池的水加热，让陆续到来的       客人来洗澡；短连接相当于每个客人离开后停止加热，等下一个客人来以后再开始加热。**_<a/>

**2.长连接的优缺点**

建立连接的过程比较复杂，所以尽量减少建立连接的操作（类似于频繁的开机关机），尽量使用长连接，但全部使用长连接后，**因为mysql在执行过程临时使用的内存是管理在连接对象里，只有在连接断开时候才能释放**，所以占用内存涨的特别快，如果长连接累积下来会导致内存占用太大，被系统强制杀掉（OOM），从现象看就是**mysql异常重启**。

_**解决方式如下**_：

1. 定期断开长连接，按照时间或者事件（比如程序判断执行过一个占用内存的大查询）来断开和重连；
2. 如果你的mysql版本大于等于 mysql5.7，可以在每次执行完大操作后，通过mysql_reset_connection来重新初始化连接资源。

<a>**一个比方：假设来热水池洗澡的客人有胖子有瘦子，需要的热水量不同，如果是瘦子来洗澡就一直加热对应的热水池，如果是胖子，在他洗完澡离开后停止加热，释放能量，等下次胖子再来洗澡时再提供能量加热，这样不会导致热水池能量崩溃**</a>

### 查询缓存

功能定义:mysql拿到一个查询请求（select 语句）会先到查询缓存看是否执行过这个语句，历史执行语句及其结果以{查询语句：查询结果}这种key-value的形式缓存在内存。如果查询可以match到key，就会直接返回结果；如果不在，继续后面的执行阶段；执行完成后，执行结果也会append到内存里。

_**但大多数情形下，不建议使用查询缓存，弊大于利**_，原因如下：

**一个表有更新，之前建立的查询缓存就会被清空。所以对于更新压力大的数据库，查询缓存命中率非常低;除非你的业务表是一张静态表。**

**“按需使用”**:

可以将query_cache_type设置成DEMAND,对于默认的sql语句不使用查询缓存，对于确定要使用查询缓存的语句，可以通过SQL_CACHE显式指定：
`mysql>sselect SQL_CACHE * from T where ID=10;`

### 分析器

功能定义:如果没有命中查询缓存，开始执行语句。

* step1，对你输入的由多个字符串和空格组成的sql语句进行**“词法分析”**，比如从输入的“select”这个关键字识别出是查询语句等；
* step2, 做完词法分析之后，会进行**“语法分析”**，语法分析器会根据语法规则，判定你输入的sql语句是否满足mysql语法。

比如下面的查询语句,select少了一个‘s’：

`mysql> elect * from trip_points where stop_id = 11;
ERROR 1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 'elect * from trip_points where stop_id = 11' at line 1’.`

一般语法错误会提示第一个出现错误的位置，所以关注紧接“use near”的内容即可。

### 优化器 

优化器是在表里面有多个索引时决定使用哪个索引；或者一个语句有多个表join时决定各个表的连接顺序。比如执行语句：

`mysql>select * from ti join t2 using(ID) where t1.c=10 and t2.d=20`

* 即可以先从表t1取出t1.c=10的记录的行，根据ID关联到t2，再判断t2.d=20的行是否存在；
* 也可以先从表t2取出t2.d=20的记录的行，根据ID关联到t1，再判断t1.c=10的行是否存在；

两种方式执行的逻辑结果是一致的，但效率不同，优化器的作用在于决定选择使用哪一个方案。

优化器阶段完成后，语句就进入执行器阶段。

### 执行器

执行步骤：1.首先判断你是否有对这个表T执行查询的权限，如果没有就返回没有权限的报错；2.如果有权限，执行器就会打开表，根据表的引擎定义，使用引擎提供的接口。比如：

`select * from T where ID=10;`

执行器的流程如下：
1. 调用InnoDB引擎接口取表的第一行，判断ID是否为10，如果不是就跳过，如果是，存入结果集；
2. 调用引擎接口取“下一行”，重复相同判断逻辑，直到最后一行；
3. 执行器将上述遍历过程中满足条件的行组成的记录集作为结果集返还给客户端。

你会在数据库的慢查询日志中看到一个rows_examined的字段，表示这个语句执行过程中扫描了多少行。


## 日志系统：一条sql更新语句是如何执行的？

从一个表的一条更新语句说起，这个表有一个主键ID和一个整型字段c：

`create table T(ID int primary key, c int);`

如果要讲ID=2这一行的值加1，SQL语句就可以这么写：

`mysql>update table T set c=c+1 where ID=2;`

更新流程也会像查询流程一样走“连接器、分析器、优化器、执行器”直到存储引擎。与查询流程不同的是，更新流程还涉及两个重要的日志模块：

_**redo log(重做日志）和binlog(归档日志）**_

#### 重要的日志模块：redo log 

<a> _**一个比方：假设你是一个饭店老板，很多客人来店里吃饭喝酒，有的人会进行赊账，那么当客人多很忙的时候，最好的操作方式是，先把赊账用粉笔记在小黑板上，等打烊以后再把账本翻出来核算。**_</a>

在mysql里也有同样的问题，如果每一次更新操作都写进磁盘，磁盘找到对应的那条记录，然后再更新，整个过程的IO升本、查找成本很高，为了解决这个问题，mysql的设计者使用了WAL技术，全称是Write-Ahead Logging，关键点就是_**先写进日志，再写进磁盘**_。

具体过程如下：
1. 当一条记录需要更新的时候，InnoDB引擎就会先把记录写到 redo log（小黑板）里面，并更新内存（查询缓存会全部清空），这时更新完成。
2. InnoDB会在适当的时候，将这个操作记录更新到磁盘，往往在系统空闲时候做这个更新，好比餐厅老板打烊以后做的事情。

**清除机制**：

InnoDB的redo log是固定大小的，比如可以配置为一组4个文件，每个文件1G，总共4G，从头开始写，写到末尾再回到开头循环写，如下图所示。其中：

1. write pos是记录当前写入位置，一边写一边后移，写到第3号文件末尾就回到0号文件开头；
2. checkpoint是当前要擦除的位置（顺时针擦除，所以从write pos的位置到checkpoint当前位置之间是空白可写入的），也是往后推移并且循环的，擦除前要把记录更新到数据文件。
3. 如果write pos赶上了checkpoint,表示redo log已满，这时候就不能执行新的更新，要停下来擦掉一些记录，把checkpoint推进一下。

_**有了redo log，InnoDB就可以保证即使数据库发生异常重启，提交的记录也不会丢失，这个能力称之为crash-safe**_

<img src="figure/redolog.jpg" width="400" hegiht="300" align=left />

#### 重要的日志模块：binlog 

**定义**

binlog是记录所有数据库表结构变更（例如CREATE、ALTER TABLE…）以及表数据修改（INSERT、UPDATE、DELETE…）的二进制日志。binlog不会记录SELECT和SHOW这类操作，因为这类操作对数据本身并没有修改，但你可以通过查询通用日志来查看MySQL执行过的所有语句。

mysql 整体来看，分为2块：

1. server层，主要做的就是mysql功能层面的事情，server层也有自己的日志，称为binlog(归档日志）；
2. 存储层，负责存储相关事宜，redo log是InnoDB引擎特有的日志。

最开始的时候，mysql里并没有InnoDB引擎，mysql自带的引擎是MyISAM，但是它没有crash-safe能力，binlog日志只能用于归档。而InnoDB是另一家公司以插件的形式引入mysql的，为了增加crash-safe能力,InnoDB使用另一套日志系统，即redo log。

redo log与binlog的不同：
1. redo log是InnoDB特有的；binlog是mysql的server层特有的，所有存储引擎都可以用；
2. redo log是物理日志，记录的是“在某个数据页上做了什么修改”；binlog是逻辑日志，记录的是这个语句的原始逻辑，比如“给ID为2的行的字段c加1”；
3. redo log是循环写的，空间固定会用完；binlog是可以追加写的，不会覆盖。

有了对两种日志的理解，我们再来看执行器和InnoDB引擎在执行简单的update操作时的内部流程，如下图所示：

<img src="figure/redolog&binlog.jpg" width="400" hegiht="300" align=left />

_**备注：浅色框表示是在InnoDB内部执行的，深色框是在执行器中执行的**_

#### 两阶段提交

上面流程图最后三步，将redo log拆成两个步骤：prepare和commit,中间安插了binlog写入操作，这就是“两阶段操作”。

_**两阶段操作的主要目的是为了让redo log与binlog的逻辑保持一致性。**_

**两阶段提交的必要性**——由于 redo log与binlog是独立的逻辑，分别存在于server和InnoDB，如果不用两阶段提交，先写完其中之一，再写另外一个，无法实现逻辑的一致：

1.**先写redo log后写binlog**。假设在redo log写完binlog还没有写完的时候，mysql异常重启，由于redo log写完crash-save的特点数据依旧可以恢复，所以恢复后这一行c的值是1。

但是由于binlog没写完，binlog里没有记录这个update语句，因此备份日志的时候，存起来的binlog里没有这条语句。系统的binlog会定期做备份（频率可以是天或者周），所以需要binlog来恢复临时库时，由于语句的binlog丢失，临时库会少了这一次更新，恢复出来临时库c的值还是0，与原库的值不同，逻辑不一致；

2.**先写binlog后写redo log**。如果binlog之后写redo log，由于redo log没写，崩溃恢复之后事务无效，所以c还是0。但是按照binlog日志记录了update的语句，恢复的时候临时库里c的值为1，和原库的值不同，引起逻辑不一致。

可以看到，_**如果不使用“两阶段提交”，那么数据库的状态就有可能和它用的日志恢复出来的库状态不一致。**_

## 事务隔离：为什么你改了我还看不见？

事务要保证一组数据库操作，要么全部成功，要么全部失败。在mysql里，事务是在存储引擎层来实现的，但并不是所有的引擎都支持事务，比如MyISAM，这也是它被InnoDB代替的重要原因之一（另一个原因应该是没有redo log功能）。

#### 隔离性与隔离级别

事务有4要素——ACID：

1. Atomicity(原子性）
2. Consistency（一致性）
3. Isolation（隔离性）
4. Durability（持久性）

我们今天主要介绍_**隔离性**_。

当数据库上有多个事务同时执行的时候，就可能出现脏读（dirty read）、不可重复读（non reapeattable read）、幻读（phantom read）的问题，为了解决这些问题，就有了**“隔离级别”**的概念。

关于隔离，隔离得越严实，效率就会越低，因此大多时候，我们要在二者之间寻找一个平衡点。sql标准的事务隔离级别包括：

1. 读未提交（read uncommitted）
2. 读提交（read committed）
3. 可重复读（repeatable read）
4. 串行化（serializable）

下面通过一个示例来给大家介绍四个概念的定义和区别。

假设现在通过下面语句创建了一个表T，只有一列，其中一行的值是1：

`mysql> creat table T(c int) engine=InnoDB;
 mysql> insert into T(c) values(1);`
 
下面的A和B是按照时间顺序执行的两个事物的行为。

<img src='figure/隔离级别.jpg' width='400' height='300' align=left />

| 隔离级别 | 读未提交 | 读提交 | 可重复读 | 串行化|
| ------- |:------| :------ | :------ | :------ | 
| 定义 | 一个事务还未提交时，它做的变更就能别的事务看到 | 一个事务提交之后，它做的变才会被别的事务看到 | 一个事务在执行过程中看到的事务总是跟这个事务在启动时看到的事务是一致的 | 后访问的事务要等前一个事务执行完成，才能继续执行 | 
| (V1,V2,V3) |(2,2,2)| (1,2,2) | (1,1,2) | (1,1,2) |
| 案例解释 | 因为B虽然还未提交，但是结果可以被“读到” | B提交后A才能看到 | V2是1的原因是，事务在执行期间看到的数据必须前后一致 | 事务B在执行“将1改为2”的时候会被锁住直到事务A提交 | 

Oracle数据库默认的隔离级别是 read committed，因此从它迁移到mysql上的应用，为了保证隔离级别一致，一定要记得更改mysql的隔离级别保持一致。

可以通过 `show variable like 'transaction_isolation'`来查看当前值。

<img src='figure/查看隔离级别.png' width='400' height='300' align=left />

#### 事务隔离的实现

在mysql中，每条记录在更新的时候，都会同时记录一条回滚操作，记录上的最新值都会通过回滚操作，得到前一个状态的值。这些都通过回滚日志的方式保留下来。

长事务意味着系统里会存在很老的事务视图，在事务提交之前，数据库里对应的回滚记录都必须保留，这就会导致大量占用存储空间。除了对回滚段的影响，长事务还占用锁资源，也可能拖垮整个库，所以要尽量避免长事务。

## 深入浅出索引

索引的出现其实就是为了提高数据查询的效率，就像书的目录一样。对于数据库的表而言，索引其实就是它的“目录”。

### 索引常见模型

 #### 哈希表
 
 哈希表是一种以key-value存储数据的结构，我们只要输入待查找的键即key，就可以找到其对应的value。哈希的设计思路是**把key放在数组里，一个哈希函数把key映射到一个确定的位置，然后把key对应的value放在这个位置。**

有可能多个key经过哈希函数换算，会对应同一个位置，处理这种情况的方法是，拉出一个_**链表**_。

假设，你维护一个身份证信息和姓名的表，需要根据身份证号查询对应名字，这时对应的哈希索引示意图如下：

<img src='figure/哈希查询.jpg' width='500'  height='400'  align=left />

**哈希表的优缺点**：

    * 优点：图中四个card_id的值（即key在数组中的index）不是递增的，这样的好处（相比有序列表）是增加新的user时速度很快，只需要往后追加；
    * 缺点：因为key在数组中的index不是有序的，所以哈希索引做区间查询速度很慢。
    
如果要找身份证在[ID_card_X,ID_card_Y]这个区间的所有用户，要对数组进行遍历。所以，**哈希表**这种结构适用于只有等值查询的场景，比如Memcached以及其他一些nosql引擎。

#### 有序数组

**有序列表在等值查询和范围查询的场景中性能都非常优秀。**上面的例子如果使用数组来实现，示意图如下：

<img src='figure/有序数组.jpg' width='500'  height='400'  align=left />

**有序数组的优缺点**：

    * 优点：假设身份证号没有重复，数组是按照身份证号递增顺序保存的，如果要查id_card_n2对应的名字，用二分法就可以快速得到，时间复杂度是O(log(N))。同时，有序结构支持范围查询，假设查找身份证号位于[a,b]区间的user，可以先用二分法找到a，然后向右遍历，直到查到b，退出循环。
    
    * 缺点：在需要更新数据的时候，你往中间插入一个记录必须挪动后面所有的记录，成本太高。
    
所以，**有序数组只适用于静态存储引擎**，比如你要保存的是2017年某个城市的人口信息，这类不会再修改的数据。   

#### 搜索树

上面的例子用二叉搜索树来实现，示意图如下：

<img src='figure/二叉树.jpg' width='500'  height='400'  align=left />

**二叉搜索树的特点**：每个节点的 左儿子<父节点<右儿子。

如果要查找某个身份证号，时间复杂度是O(log(N))，为了维持这棵树的平衡二叉树，更新的时间复杂度也是O(log(N))。

树可以由二叉，也可以有多叉，二叉树的搜索效率是最高的，但大多数数据库存储并不使用二叉树，原因是，**索引不止存在于内存中，还要写到磁盘上。**
一棵100万节点的平衡二叉树，树高为20，一次查询可能要访问20个数据块，从磁盘读取需要20个10ms（随机读一个数据块的时间）的时间，查询太慢。

**为了尽量少读磁盘，我们要使用“N”叉树。**以innoDB为例，这个N差不多是1200。树高为4的时候，可以存1200^3次方，17亿。查询一个值最多需要访问3次磁盘，树的第二层也有很大概率在内存中，访问磁盘平均次数更少了。

N叉树在读写上的性能优点，以及适配磁盘的访问模式，已经被广泛用于数据库引擎中。个人理解是，**多叉树相当于平衡了哈希表和有序数组的优缺点**。

**每碰到一个数据库，我们要先关注它的数据模型，这样才能从理论上分析出这个数据库的适用场景。**

#### innoDB的索引模型

mysql中索引是在引擎层实现的，以innoDB为例，分析下索引模型。

在innoDB中，表都是根据主键顺序以索引的形式存放，这种存储方式的表称为[索引组织表](https://www.cnblogs.com/nieliu/archive/2012/05/04/2482223.html)。innoDB使用了[B+树](https://posts.careerengine.us/p/59688d4fa870fd7b023783fb)索引模型，所以数据都是存储在B+树中。

每个索引列在innoDB中对应一棵B+树。

**自增长主键**是指自动增长主键上的值，建表语句定义如下：`NOT NULL PRIMARY KEY AUTO_INCREMENT`。系统会获取当前主键最大值加1作为下一条记录的主键值。自增长主键的插入数据模式，符合我们提到的递增插入的情景，每次插入一条新纪录，都是**追加操作**，不涉及挪动其他记录，也不会触发叶子节点的分裂（同一个节点下的叶子因为该数据块已满，导致部分叶子需要分页的情形）。

假设有一列是身份证号，从存储空间角度出发，由于每个非主键的列作为索引的叶子节点上都是主键的值。如果用身份证号作为主键，那么二级索引（非主键索引）的叶子节点占用约20个字节，如果用整型做主键，则二级索引的叶子节点只需要4个字节。

**显然，主键长度越小，普通索引的叶子节点就越小，普通索引占用的空间也就越小。**

所以，从性能和存储空间上考量，自增长主键往往是更好的选择。

<img src='figure/索引模型.jpg' width='500'  height='400'  align=left />

### 覆盖索引

在下面这个表T中，如果执行`select * from T where k between 3 and 5`,需要执行几次树的搜索操作，会扫描多少行？

`mysql> create table T (ID int primary key,
 k int NOT NULL DEFAULT 0, 
 s varchar(16) NOT NULL DEFAULT '',
 index k(k))
 engine=InnoDB;
 insert into T values(100,1, 'aa'),(200,2,'bb'),(300,3,'cc'), (500,5,'ee'), (600,6,'ff'),(700,7,gg')`

这条查询语句执行流程：
    1.在k索引树上找到k=3的记录，取得ID=300;
    2.再到ID索引树上查到ID=300对应的R3；
    3.在k索引树上找到k=5的记录，取得ID=500;
    4.再回到ID索引树上查到ID=500对应的R4；
    5.在k索引树上取下一个值k=6，不满足条件，循环结束。

在这个过程中，步骤2、4**回到主键索引树搜索的过程，我们称之为回表**。上面的查询读了k索引树的3条记录，回表两次。

这个例子中，_**由于查询结果所需要的数据只有主键索引上有，不得不回表**_。有没有可能经过索引优化，避免这个过程？

如果执行的语句是`select ID from T where k between 3 and 5`,这时只需要查ID的值，而ID的值已经在k索引树上了，因此可以直接提供查询结果，不需要回表。即在查询过程中，索引k已经“覆盖了”我们的查询需求，我们称为**覆盖索引。**

_**由于覆盖索引可以减少树的搜索次数，显著提高查询性能，索引覆盖索引是常用的性能优化手段。**_

如果现在有一个高频需求，要根据市民身份证号查询姓名，则（身份证号，姓名）这个**联合索引**可以在高频请求上用到覆盖索引，不需要回表，减少语句执行时间。

### 最左前缀原则

假设市民表定义如下：

`CREATE TABLE 'tuser' (
 'id' int(11) NOT NULL,
 'id_card' varchar(32) DEFAULT NULL,
 'name' varchar(32) DEFAULT NULL,
 'age' int(11) DEFAULT NULL,
 'ismale' tinyint(1) DEFAULT NULL,
 PRIMARY KEY ('id'),
 KEY 'id_card' ('id_card'),
 KEY 'name_age' ('name','age')
) ENGINE=InnoDB`

**B+树这种索引结构，可以利用索引的“最左前缀”来定位记录。**

为了直观说明，以（name，age）这个联合索引来分析。

<img src='figure/联合索引.jpg' width='500'  height='400'  align=left />

如果你要查name第一个字是“张”的人，你的sql语句条件是“where name like '张%'”。这时，你也能用上这个联合索引（因为其支持最左前缀原则）。**最左前缀可以是联合索引的最左N个字段，也可以是字符串索引的最左M个字符。**

**建立联合索引的时候，如何安排索引内的字段顺序**：

   * 索引的复用能力：因为支持最左前缀，所以如果已经有了（a,b）这个联合索引，就不需要单独在a上建立索引了，**即如果可以通过调整联合索引的顺序，可以少维护一个索引，优先考虑这个顺序。**
   
    
   * 节省空间：比如上面这张表，name的字段是比age字段大的，建议你创建（name，age）这个联合索引以及（age）的单字段索引。

###  索引下推
在索引内部就判断了索引对应的行的部分值，直接判断，不需要回表。

### 课后小问题
有一张表，结构如下：

`CREATE TABLE 'geek'(
 'a' int(11) NOT NULL,
 'b' int(11) NOT NULL,
 'c' int(11) NOT NULL,
 'd' int(11) NOT NULL,
 PRIMARY KEY ('a', 'b'),
 KEY 'c' ('c'),
 KEY 'ca' ('c', 'a'),
 KEY 'cb' ('c', 'b')
 )ENGINE=InnoDB `

同事告诉小王，由于历史原因，需要a、b做联合主键。小王纳闷：

    既然主键已经包含了a、b两个字段，意味着单独在字段c上创建一个索引，就已经包含了三个字段，为何要创建'ca'、'cb'两个索引？
 同事告诉他，是因为业务里有这样两种语句：
 
 `1 select * from geek where c=N order by a limit 1;
 2 select * from geek where c=N order by b limit 1;`

**参考答案：ca可以去掉，cb要保留。原因如下：**
1. 对于语句1，按照 c=N 会自动定位到主键 (a, b)，不用特意再创建ca;
2. 对于语句2，如果只有c单个字段的索引，自动定位到主键 (a, b),但主键的顺序与order by b的顺序不一致（不满足最左前缀原则），所以cb索引可以排序优化。

## mysql锁

数据库锁设计的初衷是为了处理并发问题，锁是用来合理控制资源访问规则（排队机制）的重要数据结构。根据加锁的范围，mysql里面的锁按照范围从大到小可以分为：**全局锁、表级锁、行锁。**

下面主要介绍碰到锁时的现象和背后原理。

### 全局锁

**定义：对整个数据库实例加锁。**当你需要让整个库处于只读状态的时候，可以使用 Flush tables with read lock(FTWRL) 来加全局锁。使用这个命令后，其他线程的以下语句都会被阻塞（要排队）：

1. 数据的更新语句（增删改）；
2. 数据定义语句（建表、修改表结构等）；
3. 更新类事务的提交语句。

**全局锁的典型使用场景是，做全库逻辑备份。**


### 表级锁

mysql 有两种表级锁： **表锁和元数据锁（meta data lock, MDL）。**

**表锁**：

**表锁的语法是 lock tables ... read/write。**可以用unlock tables 主动释放锁，或者客户端断开的时候自动释放。需要注意，**lock tables 语法除了限制别的线程的读写外，也限定了本线程之后的操作对象。**

**MDL（metadata lock）:**

MDL 不需要显式使用，访问一个表的时候自动加上。**MDL的作用在于保障读写的正确性。在mysql5.5 版本引入了MDL，当对一个表做增删改查（只是针对行数据，不包含更改表结构）的时候，加MDL读锁（在执行更新语句时，读锁的作用是让读的操作排队，等所有的更新语句完成后，才释放锁，可以读）；当要对表做结构变更操作的时候，加 MDL 写锁（同样，结构变更完成后，才允许写操作）：**

1. 不同线程的读锁之间不互斥，所以可以有多个线程同时对一张表进行更新操作；
2. 读锁与写锁，写锁之间是互斥的，用来保证表结构操作的安全性（表结构的变更必须是串行的，不可以并行，所以写锁是互斥的）。

**如何安全地给小表加字段？**

* 首先，要解决长事务，事务不提交，会一直占着MDL锁；
* 其次，如果更新的操作很频繁，但你不得不加一个字段（变更结构），理想的机制是，在 alter table 语句里设定等待时间，如果等待时间内可以拿到MDL写锁最好，拿不到可以先放弃，不要阻塞后面的业务语句，之后DBA再重试这个命令。

### 行锁

mysql 的行锁是在引擎层由各个存储引擎去自己实现的，，并不是所有的引擎不支持行锁，比如MylSAM，这也是它被InnoDB替代的原因。不支持行锁意味着只能通过表锁来维护，一张表任一个时刻只能有一个更新执行，影响到业务的并发度。（意味着本来可以设立多个窗口排队取钱，现在只允许一个窗口）。

**行锁就是针对数据表中行记录的锁，比如事务A更新了一行，事务B也要更新同一行，必须等A的操作完成后才能进行。**

**两阶段锁：**

两阶段封锁强调的是“加锁（增长阶段，growing phase）和解锁（缩减阶段，shrinking phase）这两项操作，且每项操作各自为一个阶段”，这就是说不管同一个事务内需要在多少个数据项上加锁，那么所有的加锁操作都只能在同一个阶段完成，在这个阶段内，不允许对对已经加锁的数据项进行解锁操作，即加锁和解锁操作不能交叉执行（同一个事务内）。

假设顾客A要在影院B购买电影票，需要以下操作：

1. 从A账户余额扣除电影票价；
2. 给影院B账户余额增加电影票价；
3. 记录一条交易日志。

如果此时有顾客C也要在影院B购买电影票。按照“两阶段锁”的特点，两个事务冲突的部分是语句2，所以为了减少行锁时间，应该按照3、1、2的顺序进行提交，最大程度减少了事务之间的锁等待，提升并发度。

**死锁和死锁检测：**

并发系统中出现循环资源依赖时，涉及的线程都在等别的线程释放资源，导致这几个线程进入无限等待的状态，称为死锁。有两种解决策略：

1. 直接进入等待，直到超时。InnoDB中，设置这个值的参数innodb_lock_wait_timeout默认值为50s，这个等待时间无法接受；
2. 发起死锁检测，发现后主动回滚到死锁链条中某一个事务，让其他事务得以继续执行。对于每个新来的线程，检测判断会不会由自己的加入导致死锁的时间复杂度是 O(n) ，假设1000个并发线程同时更新一行，死锁检测操作是100万量级，这期间要消耗大量CPU资源。

解决这种热点行的死锁问题思路：

1. 在server端控制并发度，对于相同行更新，进入引擎前排队；
2. 将热点行改成逻辑上的多行来减少锁冲突。每次随机选择一行记录进行更新，减少冲突，并行的思路来解决问题。